### **Tabla Analítica**

In [0]:
%sql
-- ================================
-- Crear tabla agregada gold_sales_daily_summary (con TAX recalculado)
-- ================================

CREATE OR REPLACE TABLE workspace.operations.gold_sales_daily_summary AS
SELECT
    -- Dimensiones de fecha
    f.BUSINESS_DATE,
    YEAR(f.BUSINESS_DATE) AS YEAR,
    MONTH(f.BUSINESS_DATE) AS MONTH,
    DAY(f.BUSINESS_DATE) AS DAY_OF_MONTH,
    DATE_FORMAT(f.BUSINESS_DATE, 'EEEE') AS DAY_OF_WEEK,

    -- Dimensiones de tienda
    o.STORE_ID,
    o.STORE_NAME,
    o.CITY,
    o.REGION_NAME,
    o.FORMAT_NAME,

    -- Dimensiones de producto
    p.ITEM AS PRODUCT_CODE,
    p.ITEM_DESC AS PRODUCT_DESC,
    p.DEPT_NAME AS DEPARTMENT,
    p.CLASS_NAME AS CLASS,
    p.BRAND_NAME AS BRAND,

    -- Dimensión proveedor
    s.SUP_NAME AS SUPPLIER_NAME,

    -- Dimensión canal
    c.CHANNEL_NAME AS CHANNEL_NAME,
    c.CHANNEL_TYPE_NAME AS CHANNEL_TYPE,

    -- Dimensión moneda
    m.CURRENCY_ABR AS CURRENCY,

    -- Métricas de negocio recalculadas
    SUM(f.TOTAL_SALE) AS TOTAL_SALE,
    ROUND(SUM(f.TOTAL_SALE * 0.16), 2) AS TOTAL_TAX,                       -- TAX recalculado
    ROUND(SUM(f.TOTAL_SALE + (f.TOTAL_SALE * 0.16)), 2) AS TOTAL_SALE_TAX,  -- TOTAL_SALE_TAX recalculado
    SUM(f.TOTAL_COST) AS TOTAL_COST,
    SUM(f.TOTAL_SALE - f.TOTAL_COST) AS GROSS_MARGIN,
    CASE WHEN SUM(f.TOTAL_SALE) = 0 THEN 0
         ELSE ROUND(SUM(f.TOTAL_SALE - f.TOTAL_COST)/SUM(f.TOTAL_SALE)*100, 2) END AS PCT_GROSS_MARGIN,
    SUM(f.UNITS) AS TOTAL_UNITS,
    COUNT(f.Q_TRANS) AS TOTAL_TRANSACTIONS,
    CASE WHEN COUNT(f.Q_TRANS) = 0 THEN 0
         ELSE ROUND(SUM(f.TOTAL_SALE + (f.TOTAL_SALE * 0.16))/COUNT(f.Q_TRANS), 2) END AS AVG_TICKET,
    CASE WHEN COUNT(f.Q_TRANS) = 0 THEN 0
         ELSE ROUND(SUM(f.UNITS)/COUNT(f.Q_TRANS), 2) END AS UNITS_PER_TRANSACTION

FROM workspace.operations.gold_fact_sales_product f
INNER JOIN workspace.operations.gold_dim_product p
    ON f.SK_PRODUCT = p.SK_PRODUCT
INNER JOIN workspace.operations.gold_dim_organization o
    ON f.SK_ORGANIZATION = o.SK_ORGANIZATION
INNER JOIN workspace.operations.gold_dim_supplier s
    ON f.SK_SUPPLIER = s.SK_SUPPLIER
INNER JOIN workspace.operations.gold_dim_channel c
    ON f.SK_CHANNEL = c.SK_CHANNEL
INNER JOIN workspace.operations.gold_dim_currency m
    ON f.SK_CURRENCY = m.SK_CURRENCY

GROUP BY
    f.BUSINESS_DATE,
    o.STORE_ID, o.STORE_NAME, o.CITY, o.REGION_NAME, o.FORMAT_NAME,
    p.ITEM, p.ITEM_DESC, p.DEPT_NAME, p.CLASS_NAME, p.BRAND_NAME,
    s.SUP_NAME,
    c.CHANNEL_NAME, c.CHANNEL_TYPE_NAME,
    m.CURRENCY_ABR;


In [0]:
%sql
SELECT COUNT(*) AS total_registros
FROM workspace.operations.gold_sales_daily_summary;

In [0]:
%sql
SELECT COUNT(*) AS total_registros
FROM workspace.operations.gold_fact_sales_product;

In [0]:
%sql
SELECT 
    (SELECT ROUND(SUM(TOTAL_SALE),2) FROM workspace.operations.gold_fact_sales_product) AS total_sale_original,
    (SELECT ROUND(SUM(TOTAL_SALE),2) FROM workspace.operations.gold_sales_daily_summary) AS total_sale_tabla_analitica;

In [0]:
%sql
SELECT
    COUNT(*) AS total_registros,
    SUM(CASE WHEN BUSINESS_DATE IS NULL THEN 1 ELSE 0 END) AS null_business_date,
    SUM(CASE WHEN STORE_ID IS NULL THEN 1 ELSE 0 END) AS null_store_id,
    SUM(CASE WHEN PRODUCT_CODE IS NULL THEN 1 ELSE 0 END) AS null_product_code,
    SUM(CASE WHEN TOTAL_SALE IS NULL THEN 1 ELSE 0 END) AS null_total_sale,
    SUM(CASE WHEN TOTAL_SALE_TAX IS NULL THEN 1 ELSE 0 END) AS null_total_sale_tax,
    SUM(CASE WHEN TOTAL_UNITS IS NULL THEN 1 ELSE 0 END) AS null_total_units,
    SUM(CASE WHEN TOTAL_TRANSACTIONS IS NULL THEN 1 ELSE 0 END) AS null_total_transactions
FROM workspace.operations.gold_sales_daily_summary;

In [0]:
%sql
-- ===============================
-- Ejemplo automático de filas que se fusionan en la tabla agregada
-- ===============================

SELECT *
FROM workspace.operations.gold_fact_sales_product
WHERE (BUSINESS_DATE, SK_ORGANIZATION, SK_PRODUCT, SK_SUPPLIER, SK_CHANNEL, SK_CURRENCY) IN (
    SELECT BUSINESS_DATE, SK_ORGANIZATION, SK_PRODUCT, SK_SUPPLIER, SK_CHANNEL, SK_CURRENCY
    FROM workspace.operations.gold_fact_sales_product
    GROUP BY BUSINESS_DATE, SK_ORGANIZATION, SK_PRODUCT, SK_SUPPLIER, SK_CHANNEL, SK_CURRENCY
    HAVING COUNT(*) > 1
)
ORDER BY BUSINESS_DATE, SK_ORGANIZATION, SK_PRODUCT
LIMIT 10;
